In [1]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz
%load_ext line_profiler
# turn off autoreload so that we can use the old model 
# when editing the current project

from coeditor.common import *
import os
from coeditor.model import RetrievalEditorModel, AttentionMode, DecodingArgs
from coeditor.service import EditPredictionService, ChangeDetector
from coeditor.c3problem import C3GeneratorCache

os.chdir(proj_root())

In [2]:
# target_dir = proj_root() / "../SPOT-copy/"
target_dir = proj_root()
model_name = "coeditor-xl-c3-dropout-v1.6-resumed"
model_path = get_model_dir(True) / model_name
model = RetrievalEditorModel.load(model_path)
model.to("cuda:0")
detector = ChangeDetector(target_dir)
service = EditPredictionService(
    detector,
    model,
    dec_args = DecodingArgs(do_sample=False, num_beams=6, length_penalty=0.0)
    # dec_args = DecodingArgs(do_sample=True, top_p=0.95, marginalize_samples=50)
)


In [3]:
model.push_to_hub(f"MrVPlusOne/{model_name}")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MrVPlusOne/coeditor-xl-c3-dropout-v1.6-resumed/commit/09040cdc1a676199b31c944679f7ae2116404f17', commit_message='Upload model', commit_description='', oid='09040cdc1a676199b31c944679f7ae2116404f17', pr_url=None, pr_revision=None, pr_num=None)

In [4]:
# service.dec_args = DecodingArgs(do_sample=True, top_p=0.9, marginalize_samples=25)
# service.dec_args = DecodingArgs(do_sample=False, num_beams=8, length_penalty=0.0)
service.tlogger.clear()
response = service.suggest_edit(to_rel_path("scripts/train_model.py"), 91)
print(response)
service.tlogger.as_dataframe()

<coroutine object EditPredictionService.suggest_async at 0x7fe2921429e0>


,name,count,avg_time,total_time


In [ ]:
from pprint import pprint
pprint(service.detector.analyzer.error_counts)